In [1]:
import json
import pandas as pd
from arize.otel import register
from openinference.instrumentation.openai import OpenAIInstrumentor
from openai import AzureOpenAI
from phoenix.evals import OpenAIModel, llm_classify

c:\Pranav\GenaiAssurance_ClaimDemoApp\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load Arize credentials
with open(r"C:\Pranav\arize_creds.json") as f:
    arize_creds = json.load(f)

# Load Azure OpenAI credentials
with open(r"C:\Pranav\azure_openai_creds_new.json") as f:
    azure_creds = json.load(f)

In [3]:
# arize_creds = {
#     "api_key" : "ak-9925985c-529b-4060-b6bd-0fe2f919bac2-iuvJLc2IdtZZa_qqixKF2bqlx26Re8lh",
#     "space_id" : "U3BhY2U6MTc5NTY6WDlaZw==",
#     "project_name" : "insurance_multi_agent_project",
#     "user": "insurance_multi_agent_project"
# }

In [21]:
claim_input = {
    "name": "Valid Medical",
    "policy_id": "HLT-2001",
    "policy_start": "2025-01-01",
    "policy_end": "2025-12-31",
    "claim_type": "Health Insurance (Major Medical Plan)",
    "accident_date": "2025-05-15",
    "claim_amount": 12000,
    "is_valid_claim": True,
    "covered_rules": "{'document_verification_agent': ['submitted_documents_completeness: All required documents, such as medical bills, medical records, and proof of payment, must be provided.', 'authenticity_check: Submitted documents must be genuine and free from tampering or forgery.', 'duplicate_document_check: Ensure that no duplicate bills or records are submitted for the same service.', 'provider_details_verification: Verify that provider information (e.g., hospital or surgeon) on submitted documents matches official records.', 'service_date_verification: Confirm that the service dates on documents are clearly stated and align with the claimed event.']}",
    "notes": "['document_verification_agent - submitted_documents_completeness: All required documents have likely been provided, as inferred from the valid claim status.', 'document_verification_agent - authenticity_check: Submitted documents are genuine, as inferred from the valid claim status.', 'document_verification_agent - duplicate_document_check: No duplicate bills or records have been submitted, as inferred from the valid claim status.', 'document_verification_agent - provider_details_verification: Provider information matches official records, as inferred from the valid claim status.', 'document_verification_agent - service_date_verification: Service dates on documents align with the claimed event, as inferred from the valid claim status.', \"eligibility_checker_agent - network_status_verification: The provider's network status is appropriate for the policy, as inferred from the valid claim status.\", 'eligibility_checker_agent - medical_necessity_review: The claimed procedure is medically necessary, as inferred from the valid claim status.', 'eligibility_checker_agent - claim_timing_check: The claim submission is within the allowable time frame, as inferred from the valid claim status.', 'fraud_checker_agent - duplicate_claim_review: No identical or overlapping claims have been submitted, as inferred from the valid claim status.', 'fraud_checker_agent - inconsistency_detection: No inconsistencies exist between submitted documents, as inferred from the valid claim status.', 'fraud_checker_agent - provider_verification: The listed provider exists and is valid, as inferred from the valid claim status.', \"fraud_checker_agent - service_reasonability_check: The claimed services align with the policyholder's medical history and condition, as inferred from the valid claim status.\"]",
    "document": "hospital_bill_knee_surgery.pdf",
    "document_path": "C:\\Pranav\\claim_documents\\hospital_bill_knee_surgery.pdf",
}

df_1 = pd.DataFrame([claim_input])

In [22]:
df_1.head(2)

name policy_id policy_start  policy_end  \
0  Valid Medical  HLT-2001   2025-01-01  2025-12-31   

                              claim_type accident_date  claim_amount  \
0  Health Insurance (Major Medical Plan)    2025-05-15         12000   

   is_valid_claim                                      covered_rules  \
0            True  {'document_verification_agent': ['submitted_do...   

                                               notes  \
0  ['document_verification_agent - submitted_docu...   

                         document  \
0  hospital_bill_knee_surgery.pdf   

                                       document_path  
0  C:\Pranav\claim_documents\hospital_bill_knee_s...

question, expected_tool, tool_name, args, expected_args

In [25]:
eval(df_1["covered_rules"][0])

{'document_verification_agent': ['submitted_documents_completeness: All required documents, such as medical bills, medical records, and proof of payment, must be provided.',
  'authenticity_check: Submitted documents must be genuine and free from tampering or forgery.',
  'duplicate_document_check: Ensure that no duplicate bills or records are submitted for the same service.',
  'provider_details_verification: Verify that provider information (e.g., hospital or surgeon) on submitted documents matches official records.',
  'service_date_verification: Confirm that the service dates on documents are clearly stated and align with the claimed event.']}

In [12]:
# Load Excel test cases
df = pd.read_excel(r"insurance_test_cases.xlsx")

In [13]:
df#.head(2)

goal   persona  \
0   file medical claim     angry   
1   check claim status   curious   
2   file vehicle claim  indirect   
3  download claim form    direct   

                                               query expected_tool  \
0  I was hospitalized last week! Why haven’t you ...    file_claim   
1  Can you update me on my vehicle claim for the ...  check_status   
2             My car was hit this morning, now what?    file_claim   
3       I need the claim form for medical insurance.      get_form   

                                       expected_args  
0  {"type": "medical", "incident_date": "...", "d...  
1                             {"claim_id": "123456"}  
2  {"type": "vehicle", "incident_date": "...", "d...  
3                                {"type": "medical"}

In [43]:
df["expected_args"][0]

'{"type": "medical", "incident_date": "...", "details": "..."}'

In [12]:
# Setup Arize OTEL Tracing
tracer_provider = register(
    space_id=arize_creds["space_id"],
    api_key=arize_creds["api_key"],
    project_name=arize_creds["project_name"]
)

# Setup OpenInference tracing for Azure OpenAI
OpenAIInstrumentor().instrument(tracer_provider=tracer_provider)

OpenTelemetry Tracing Details
|  Arize Project: insurance_multi_agent_project
|  Span Processor: BatchSpanProcessor
|  Collector Endpoint: otlp.arize.com
|  Transport: gRPC
|  Transport Headers: {'authorization': '****', 'api_key': '****', 'arize-space-id': '****', 'space_id': '****', 'arize-interface': '****', 'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [21]:
# Initialize Azure OpenAI Client
client = AzureOpenAI(
    api_key=azure_creds["OPENAI_API_KEY"],
    api_version=azure_creds["OPENAI_API_VERSION"],
    azure_endpoint=azure_creds["azure_endpoint"]
)

deployment = azure_creds["deployment_name"]

In [ ]:
# Define tools
tools = [
    {
        "type": "function",
        "function": {
            "name": "document_verifier_agent",
            "description": "Validates an insurance claim for the document verification step based on claim details and predefined rules.",
            "parameters": {
                "type": "object",
                "properties": {
                    "state": {
                        "type": "object",
                        "description": "State dictionary containing claim details and rules",
                        "properties": {
                            "claim_record": {
                                "type": "object",
                                "description": "Claim details including name, policy_id, policy_start, policy_end, claim_type, accident_date, claim_amount, and notes",
                                "properties": {
                                    "name": {"type": "string", "description": "Name of the claimant"},
                                    "policy_id": {"type": "string", "description": "Unique policy identifier"},
                                    "policy_start": {"type": "string", "description": "Policy start date"},
                                    "policy_end": {"type": "string", "description": "Policy end date"},
                                    "claim_type": {"type": "string", "description": "Type of claim (e.g., Auto Insurance (Collision Coverage))"},
                                    "accident_date": {"type": "string", "description": "Date of the incident"},
                                    "claim_amount": {"type": "number", "description": "Claim amount"},
                                    "notes": {
                                        "type": "array",
                                        "items": {"type": "string"},
                                        "description": "List of rule validations or inferences"
                                    }
                                },
                                "required": ["policy_id", "claim_type", "accident_date"]
                            },
                            "rules": {
                                "type": "array",
                                "items": {"type": "object"},
                                "description": "List of rules specific to document_verifier_agent for the claim type"
                            }
                        },
                        "required": ["claim_record", "rules"]
                    }
                },
                "required": ["state"]
            }
        }
    }
]

question, expected_tool, tool_name, args, expected_args

In [16]:
# Define evaluation templates
ROUTER_EVAL_TEMPLATE = """
You are evaluating if the correct function was selected for this user question.

[Question]: {question}
[LLM Tool Call]: {tool_name}
[Expected Tool]: {expected_tool}

Is the selected function correct? Respond with "correct" or "incorrect".
"""

ARGS_EVAL_TEMPLATE = """
You are evaluating if the arguments passed to a function match the expected ones.

[Function]: {tool_name}
[Actual Args]: {args}
[Expected Args]: {expected_args}

Respond with "correct" or "incorrect".
"""

In [ ]:
# Function to run prompt and return tool name + arguments
def run_prompt(query):

    response = client.chat.completions.create(
        model=deployment,
        temperature=0,
        tools=tools,
        tool_choice="required",
        messages=[{"role": "user", "content": query}]
    )
    
    tool_calls = response.choices[0].message.tool_calls
    return tool_calls[0].function.name, json.loads(tool_calls[0].function.arguments)

In [18]:
df.head(2)

goal  persona  \
0  file medical claim    angry   
1  check claim status  curious   

                                               query expected_tool  \
0  I was hospitalized last week! Why haven’t you ...    file_claim   
1  Can you update me on my vehicle claim for the ...  check_status   

                                       expected_args  
0  {"type": "medical", "incident_date": "...", "d...  
1                             {"claim_id": "123456"}

In [23]:
# Run agent over test cases and collect results
results = []

for _, row in df.iterrows():

    tool_name, args = run_prompt(row['query'])
    
    results.append({
        "goal": row['goal'],
        "query": row['query'],
        "expected_tool": row['expected_tool'],
        "expected_args": row['expected_args'],
        "tool_name": tool_name,
        "tool_args": json.dumps(args)
    })

# Create results DataFrame
response_df = pd.DataFrame(results)

# Patch column names to match templates
response_df["question"] = response_df["query"]
response_df["args"] = response_df["tool_args"]

# Save raw results
# response_df.to_csv("conversation_log.csv", index=False)

In [26]:
response_df.head(2)

goal                                              query  \
0  file medical claim  I was hospitalized last week! Why haven’t you ...   
1  check claim status  Can you update me on my vehicle claim for the ...   

  expected_tool                                      expected_args  \
0    file_claim  {"type": "medical", "incident_date": "...", "d...   
1  check_status                             {"claim_id": "123456"}   

      tool_name                tool_args  \
0  check_status  {"claim_id": "unknown"}   
1  check_status     {"claim_id": "last"}   

                                            question                     args  
0  I was hospitalized last week! Why haven’t you ...  {"claim_id": "unknown"}  
1  Can you update me on my vehicle claim for the ...     {"claim_id": "last"}

question, expected_tool, tool_name, args, expected_args

In [31]:
response_df.shape

(4, 8)

In [28]:
# Evaluation 1: Tool Selection
tool_eval = llm_classify(
    data=response_df,
    template=ROUTER_EVAL_TEMPLATE,
    model=OpenAIModel(
        model="gpt-4o",
        api_key=azure_creds["OPENAI_API_KEY"],
        api_version=azure_creds["OPENAI_API_VERSION"],
        azure_endpoint=azure_creds["azure_endpoint"],
        azure_deployment=azure_creds["deployment_name"]
    ),
    rails=["correct", "incorrect"],
    provide_explanation=True,
    concurrency=4,
)
# tool_eval.to_csv("tool_eval.csv", index=False)

🐌!! If running inside a notebook, patching the event loop with nest_asyncio will allow asynchronous eval submission, and is significantly faster. To patch the event loop, run `nest_asyncio.apply()`.
llm_classify |██████████| 4/4 (100.0%) | ⏳ 00:08<00:00 |  2.24s/it


In [32]:
tool_eval.shape

(4, 5)

In [34]:
tool_eval#.head(2)

label                                        explanation exceptions  \
0    correct  The user is asking why their claim hasn't been...         []   
1    correct  The user is asking for an update on their vehi...         []   
2  incorrect  The user's question is about what steps to tak...         []   
3    correct  The user is asking for a claim form for medica...         []   

  execution_status  execution_seconds  
0        COMPLETED           3.703496  
1        COMPLETED           1.977593  
2        COMPLETED           1.856740  
3        COMPLETED           1.427628

In [35]:
tool_eval["explanation"].to_list()

["The user is asking why their claim hasn't been processed yet, which implies they believe a claim has already been filed. The selected function, 'check_status,' is appropriate because it checks the status of an existing claim. The 'file_claim' function would only be relevant if the user were asking to initiate a new claim, which is not the case here.",
 "The user is asking for an update on their vehicle claim, which aligns with the purpose of the 'check_status' function. This function is used to check the status of claims or similar inquiries.",
 "The user's question is about what steps to take after their car was hit, which suggests they may need to file an insurance claim. The 'file_claim' function would be the appropriate choice to guide them through this process. The 'get_form' function does not directly address the user's situation.",
 "The user is asking for a claim form for medical insurance, which aligns with the purpose of the 'get_form' function. This function is appropriate

In [39]:
# Evaluation 2: Argument Accuracy
args_eval = llm_classify(
    data=response_df,
    template=ARGS_EVAL_TEMPLATE,
    model=OpenAIModel(
        model="gpt-4o",
        api_key=azure_creds["OPENAI_API_KEY"],
        api_version=azure_creds["OPENAI_API_VERSION"],
        azure_endpoint=azure_creds["azure_endpoint"],
        azure_deployment=azure_creds["deployment_name"]
    ),
    rails=["correct", "incorrect"],
    provide_explanation=True,
    concurrency=4,
)
# args_eval.to_csv("args_eval.csv", index=False)

🐌!! If running inside a notebook, patching the event loop with nest_asyncio will allow asynchronous eval submission, and is significantly faster. To patch the event loop, run `nest_asyncio.apply()`.
llm_classify |██████████| 4/4 (100.0%) | ⏳ 00:13<00:00 |  3.26s/it


In [40]:
args_eval

label                                        explanation exceptions  \
0  incorrect  The actual arguments passed to the function do...         []   
1  incorrect  The actual argument for 'claim_id' is 'last', ...         []   
2  incorrect  The actual arguments only include the 'type' k...         []   
3    correct  The actual arguments passed to the function ma...         []   

  execution_status  execution_seconds  
0        COMPLETED           3.329412  
1        COMPLETED           5.537482  
2        COMPLETED           2.282559  
3        COMPLETED           1.904673